In [3]:
"""
Assignment 10 - Word Embeddings

Skip-gram approach
Training on a corpus of Shakespeare

Test on the classification of movie reviews. 



Adda Weathers
February 23, 2025
"""

'\nAssignment 10 - Word Embeddings\n\nSkip-gram approach\nTraining on a corpus of Shakespeare and\nA corpus of contemporary language\n\nTest on the classification of movie reviews. \n\n\n\nAdda Weathers\nFebruary 23, 2025\n'

In [4]:
# Step 1: Libraries
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
# Testing to make sure these libraries installed without error
print("Libraries successfully installed.")

Libraries successfully installed.


In [5]:
# Load TensorBoard ext
%load_ext tensorboard

In [6]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [7]:
sentence = "He hath eaten me out of house and home He ate so much there was nothing left"
tokens = list(sentence.lower().split())
print(len(tokens))
print("Shakesparean Sentence here.")

17
Shakesparean Sentence here.


In [8]:
vocab, index = {}, 1  # starting index from 1
vocab['<pad>'] = 0  # padding token here
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)
print("Index and padding check")

{'<pad>': 0, 'he': 1, 'hath': 2, 'eaten': 3, 'me': 4, 'out': 5, 'of': 6, 'house': 7, 'and': 8, 'home': 9, 'ate': 10, 'so': 11, 'much': 12, 'there': 13, 'was': 14, 'nothing': 15, 'left': 16}
Index and padding check


In [9]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)
print("Creating inverse vocabulary")

{0: '<pad>', 1: 'he', 2: 'hath', 3: 'eaten', 4: 'me', 5: 'out', 6: 'of', 7: 'house', 8: 'and', 9: 'home', 10: 'ate', 11: 'so', 12: 'much', 13: 'there', 14: 'was', 15: 'nothing', 16: 'left'}
Creating inverse vocabulary


In [10]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)
print("Vectorizing the sentence")

[1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 10, 11, 12, 13, 14, 15, 16]
Vectorizing the sentence


In [11]:
print("Now we will do the skipgrams.")
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0)
print(len(positive_skip_grams))

Now we will do the skipgrams.
62


In [12]:
print("Now we will do a few positive skipgrams.")
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

Now we will do a few positive skipgrams.
(5, 4): (out, me)
(6, 8): (of, and)
(15, 13): (nothing, there)
(11, 10): (so, ate)
(2, 3): (hath, eaten)


In [13]:
print("Negative sampling for a skipgram.")
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

Negative sampling for a skipgram.
tf.Tensor([4 1 9 3], shape=(4,), dtype=int64)
['me', 'he', 'home', 'eaten']


In [14]:
print("Constructing one training example now.")
# Reduce a dimension to use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word

Constructing one training example now.


In [15]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")
print("Context and corresponding labels now.")

target_index    : 5
target_word     : out
context_indices : [4 4 1 9 3]
context_words   : ['me', 'me', 'he', 'home', 'eaten']
label           : [1 0 0 0 0]
Context and corresponding labels now.


In [16]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : 5
context : tf.Tensor([4 4 1 9 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


In [17]:
print("Compile it all into one function.")
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

Compile it all into one function.
[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [18]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

In [19]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [20]:
with open(path_to_file) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [21]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))
print("Constructing TextLineDataset")

Constructing TextLineDataset


In [22]:
print("Vectorize sentences from the corpus")
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


Vectorize sentences from the corpus


In [23]:
print("Create vocabulary")
vectorize_layer.adapt(text_ds.batch(1024))


Create vocabulary


In [24]:
print("Save created vocab for reference.")
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])


Save created vocab for reference.
['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [25]:
print("Vectorize data")
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()


Vectorize data


In [26]:
print("Obtain sequences from the dataset.")
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))


Obtain sequences from the dataset.
32777


In [27]:
print("Inspecting a few examples from the sequence we just made.")
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

Inspecting a few examples from the sequence we just made.
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


In [28]:
print("Generating training examples from sequence.")
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")


Generating training examples from sequence.


100%|███████████████████████████████████████████████████████████████████████████| 32777/32777 [01:11<00:00, 459.31it/s]




targets.shape: (65080,)
contexts.shape: (65080, 5)
labels.shape: (65080, 5)


In [29]:
print("Configuring dataset for performance.")
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

Configuring dataset for performance.
<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [30]:
print("Applying Dataset.Cache to improve performance.")
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)


Applying Dataset.Cache to improve performance.
<_PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [31]:
print("Model and training")
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots


Model and training


In [32]:
print("Defining loss function here.")
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)


Defining loss function here.


In [33]:
print("Building the model now.")
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])


Building the model now.


In [34]:
print("Callback to log training statistics.")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

Callback to log training statistics.


In [35]:
print("Training model for 20 epochs.")
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])


Training model for 20 epochs.
Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.2182 - loss: 1.6089
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - accuracy: 0.5928 - loss: 1.5888
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5931 - loss: 1.5298
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5482 - loss: 1.4415
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.5680 - loss: 1.3446
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.6038 - loss: 1.2474
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6419 - loss: 1.1568
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.6799 - loss: 1.0735
Epoch 9/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.7107 - loss: 0.9969
Epoch 10/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7400 - loss: 0.9263
Epoch 11/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 0.7672 - loss: 0.8611
Epoch 12/20
63/63 ━━━━━━━━━━━━━━━

In [36]:
print("showing accuracy and loss now.")
#docs_infra: no_execute
%tensorboard --logdir logs


showing accuracy and loss now.


Reusing TensorBoard on port 6006 (pid 17060), started 0:28:20 ago. (Use '!kill 17060' to kill it.)

In [37]:
print("Obtaining weights from the model.")
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()


Obtaining weights from the model.


In [38]:
print("Create and save vectors and metafiles.")
out_v = io.open('C:/Users/addaw/OneDrive/Desktop/vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('C:/Users/addaw/OneDrive/Desktop/metadata.tsv', 'w', encoding='utf-8')
print("Files saved successfully.")
for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()


Create and save vectors and metafiles.
Files saved successfully.


In [39]:
# File paths
vectors_file_path = 'C:/Users/addaw/OneDrive/Desktop/vectors.tsv'
metadata_file_path = 'C:/Users/addaw/OneDrive/Desktop/metadata.tsv'

# Check if the files exist at the specified location
import os
if os.path.exists(vectors_file_path) and os.path.exists(metadata_file_path):
    print("Files are located at:")
    print("Vectors file:", vectors_file_path)
    print("Metadata file:", metadata_file_path)
else:
    print("Files not found at the specified location.")


Files are located at:
Vectors file: C:/Users/addaw/OneDrive/Desktop/vectors.tsv
Metadata file: C:/Users/addaw/OneDrive/Desktop/metadata.tsv


In [44]:
import numpy as np
from gensim.models import Word2Vec

# Load the word embeddings and vocabulary from the files
#vectors_file_path = 'vectors.tsv'
#metadata_file_path = 'metadata.tsv'
vectors_file_path = 'C:/Users/addaw/OneDrive/Desktop/vectors.tsv'
metadata_file_path = 'C:/Users/addaw/OneDrive/Desktop/metadata.tsv'

# Read the vectors (embeddings) from the vectors.tsv file
word_vectors = []
with open(vectors_file_path, 'r', encoding='utf-8') as f:
    for line in f:
        word_vectors.append([float(x) for x in line.strip().split('\t')])

# Read the vocabulary (words) from the metadata.tsv file
vocab = []
with open(metadata_file_path, 'r', encoding='utf-8') as f:
    vocab = [line.strip() for line in f]

# Convert the list of word vectors into a numpy array
word_vectors = np.array(word_vectors)

# Create an empty Word2Vec model 
model = Word2Vec(vector_size=word_vectors.shape[1], window=5, min_count=1)

# Assign vocabulary and vectors manually
model.wv.index_to_key = vocab  # List of words in the vocabulary
model.wv.key_to_index = {word: idx for idx, word in enumerate(vocab)}  # Mapping from word to index
model.wv.vectors = word_vectors  # Assign the pre-trained word vectors


print(model.wv['love'])  # Example: print the vector for the word "love"



[ 0.19337258 -0.20318556 -0.17330678 -0.08985364  0.0095684  -0.21611957
 -0.06115547 -0.04213292 -0.07683181 -0.01346147  0.07778812 -0.02473289
 -0.22403194 -0.18107477  0.07811909 -0.24536036 -0.01422484 -0.24012847
 -0.08465812  0.15595421 -0.03600831  0.05431313  0.02325363  0.03042642
 -0.02321665 -0.05719719  0.10277817 -0.04997594  0.27161667  0.15970348
  0.1652903   0.10762627 -0.05198477 -0.10737839  0.20506579  0.1559882
 -0.02573442  0.2953854  -0.01240354 -0.32764032  0.09988648 -0.29425713
  0.124126    0.14545915 -0.03373169 -0.12321917  0.01384258 -0.004945
  0.06247162  0.3249174  -0.1318043   0.02927196 -0.37511235 -0.10715657
 -0.10011723  0.02760358  0.02510136 -0.2877221   0.0349504  -0.0813323
  0.38863638 -0.00752864  0.03241861  0.28781715 -0.02518696 -0.10198931
  0.17897213  0.14308754  0.27082503 -0.11057413  0.429969    0.07336318
  0.10728776  0.20441571 -0.01151988 -0.01857583  0.04492797  0.29041645
 -0.0137245   0.04686267  0.17430477  0.25495163  0.081

In [49]:
# For illustrative purposes, a very simple movie review set. Very, very basic. 
print("Now we have a small set of reviews.")
reviews = ["I loved this movie!", "It was a terrible movie.", "An amazing experience!", "This movie sucked.", "What an awful movie.", "Waste of time.", "The movie was good."]
labels = [1, 0, 1, 0, 0, 0, 1]  # 1 = positive, 0 = negative

Now we have a small set of reviews.


In [50]:
print("Convert reviews into word embeddings.")

# Function to get the vector representation of a review
def get_review_vector(review, model):
    words = review.split()  # Tokenize the review (simple split by space)
    word_vectors = []

    # Iterate over the words in the review
    for word in words:
        # If the word is in the model's vocabulary, get its vector
        if word in model.wv:
            word_vectors.append(model.wv[word])
    
    # If no words were found in the vocabulary, return a zero vector
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    
    # Return the average of all word vectors in the review
    review_vector = np.mean(word_vectors, axis=0)
    return review_vector

# Convert all reviews into vectors
review_vectors = [get_review_vector(review, model) for review in reviews]

# Check the shape of one of the review vectors (should match the vector size)
print(review_vectors[0].shape)


Convert reviews into word embeddings.
(128,)


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Convert the list of review vectors into a NumPy array
X = np.array(review_vectors)  # Features (review embeddings)
y = np.array(labels)  # Labels (positive/negative)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Logistic Regression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the classifier's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Classification Accuracy: {accuracy}")


Classification Accuracy: 0.0


In [52]:
from sklearn.metrics import classification_report

# Print a detailed classification report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0



In [53]:
"""
Results:

Zero accuracy, but we had a very small movie review dataset.
Maybe a modern text will do better.

"""

'\nResults:\n\nZero accuracy, but we had a very small movie review dataset.\nMaybe a modern text will do better.\n\n'